In [ ]:
import numpy as np

#Initialization
ACTION_SPACE = ('P', 'Q')
States=[i for i in range(1,13)]

In [ ]:
States

In [ ]:
actions = {1: ('P','Q'),
           2: ('P', 'Q'),
           3: ('P', 'Q'),
           4: ('P', 'Q'),
           5: ('P', 'Q'),
           6: ('P', 'Q'),
           7: ('P', 'Q'),
           8: ('P', 'Q'),
           9: ('P', 'Q'),
          10: ('P', 'Q')
  }

In [ ]:
probs = {
    (1,'P',2): 0.99,
    (1,'P',11): 0.01,
    (1,'Q',11): 1,
    (2,'P',3): 0.90,
    (2,'P',11): 0.10,
    (2,'Q',11):1,
    (3,'P',4): 0.80,
    (3,'P',11): 0.20,
    (3,'Q',11):1,
    (4,'P',5): 0.70,
    (4,'P',11): 0.30,
    (4,'Q',11):1,
    (5,'P',6): 0.60,
    (5,'P',11): 0.40,
    (5,'Q',11):1,
    (6,'P',7): 0.50,
    (6,'P',11): 0.50,
    (6,'Q',11):1,
    (7,'P',8): 0.40,
    (7,'P',11): 0.60,
    (7,'Q',11):1,
    (8,'P',9): 0.30,
    (8,'P',11): 0.70,
    (8,'Q',11):1,
    (9,'P',10): 0.20,
    (9,'P',11): 0.80,
    (9,'Q',11):1,
    (10,'P',12): 0.1,
    (10,'P',11): 0.9,
    (10,'Q',11):1,
  }
REWARDS={
    (1,'P',2): 10,
    (1,'P',11): 0,
    (1,'Q',11): 0,
    (2,'P',3): 50,
    (2,'P',11): -10,
    (2,'Q',11):0,
    (3,'P',4): 100,
    (3,'P',11): -60,
    (3,'Q',11):0,
    (4,'P',5): 500,
    (4,'P',11): -160,
    (4,'Q',11):0,
    (5,'P',6): 1000,
    (5,'P',11): -660,
    (5,'Q',11):0,
    (6,'P',7): 5000,
    (6,'P',11): -1660,
    (6,'Q',11):0,
    (7,'P',8): 10000,
    (7,'P',11): -6660,
    (7,'Q',11):0,
    (8,'P',9): 50000,
    (8,'P',11): -16660,
    (8,'Q',11):0,
    (9,'P',10): 100000,
    (9,'P',11): -66660,
    (9,'Q',11):0,
    (10,'P',12):500000,
    (10,'P',11):-166660,
    (10,'Q',11):0,
  }

In [ ]:
def is_terminal(s):
    return s in [11,12]

In [ ]:
transition_probs = {}
# the key is (s, a, s'), the value is the probability
# that is, transition_probs[(s, a, s')] = p(s' | s, a)
# any key NOT present will considered to be impossible (i.e. probability 0)
rewards={}
for s in States:
    if not is_terminal(s):
        for a in ACTION_SPACE:
            for s2 in States:
                if (s,a,s2) in probs:
                    transition_probs[(s, a, s2)]=probs.get((s, a, s2),0)
                else:
                    transition_probs[(s, a, s2)]=0
                if (s,a,s2) in REWARDS:
                    rewards[(s, a, s2)]=REWARDS.get((s, a, s2),0)
                else:
                    rewards[(s, a, s2)]=0

In [ ]:
transition_probs

In [ ]:
rewards

In [ ]:
### fixed policy always play ###
policy = {
    1:'P',
    2:'P',
    3:'P',
    4:'P',
    5:'P',
    6:'P',
    7:'P',
    8:'P',    
    9:'P',
    10:'P', 
  }

In [ ]:
def policy_evaluation(policy):
    # initialize V(s) = 0
    V = {}
    for s in States:
        V[s] = 0
    SMALL_ENOUGH = 1e-3 # threshold for convergence
    gamma = 1 # discount factor
    # repeat until convergence
    it = 0
    while True:
        biggest_change = 0
        for s in States:
            if is_terminal(s):
                V[s]=0
            else:
                old_v=V[s]
                new_v = 0 # we will accumulate the answer
                a=policy.get(s)
                for s2 in States:
                    # reward is a function of (s, a, s')
                    r = rewards.get((s, a, s2),0)
                    new_v += transition_probs.get((s, a, s2),0) * (r + gamma * V[s2])
                # after done getting the new value, update the value table
                V[s] = new_v
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        print("iter:", it, "biggest_change:", biggest_change)
        it += 1
        if biggest_change < SMALL_ENOUGH:
            break
    print("\n\n")
    return(V)

In [ ]:
Values=policy_evaluation(policy)
Values

In [ ]:
### fixed policy ###
policy = {
    1:'P',
    2:'P',
    3:'P',
    4:'P',
    5:'P',
    6:'Q',
    7:'Q',
    8:'Q',    
    9:'Q',
    10:'Q', 
  }

In [ ]:
Values=policy_evaluation(policy)
Values

In [ ]:
for s in actions.keys():
    policy[s] = np.random.choice(actions.get(s))
policy

In [ ]:
while True:
    gamma = 1
    # policy evaluation step - we already know how to do this!
    print(policy)
    V = policy_evaluation(policy)
    # policy improvement step
    is_policy_converged = True
    for s in States:
        if not is_terminal(s):
            old_a = policy[s]
            new_a = None
            best_value = float('-inf')
            # loop through all possible actions to find the best current action
            for a in actions.get(s):
                v = 0
                for s2 in States:
                # reward is a function of (s, a, s'), 0 if not specified
                    r = rewards.get((s, a, s2), 0)
                    v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])
                if v > best_value:
                    best_value = v
                    new_a = a
                  # new_a now represents the best action in this state
                    policy[s] = new_a
            if new_a != old_a:
                is_policy_converged = False
    if is_policy_converged:
        break

In [ ]:
V